In [ ]:
import numpy as np
import pandas as pd
from plotly import express as px

In [ ]:
import os
os.getcwd()

In [ ]:
df = pd.read_csv('reg_data/train_data.csv', index_col=0, parse_dates=['date'])
df.reset_index(drop=True, inplace=True)

In [ ]:
df.dtypes

In [ ]:
df.date[34]

In [ ]:
# .sort_values(['tic','date'])[['tic','date',]
wat = df.groupby("tic", as_index=True)['close'].ewm(com=60, adjust=True).std()
wat = wat.droplevel(0)

In [ ]:
wat.index

In [ ]:
wat

In [ ]:
df["volatility"] = wat

In [ ]:
df

In [ ]:
px.line(df[df.tic == "BA"], x = 'date', y = "volatility", color = 'tic',  )

In [ ]:
px.line(df[df.tic == "BA"], x = 'date', y = "close", color = 'tic',  )

In [ ]:
delta = 0.9836065
delta/(1-delta)
I = 1000

w = [(1-delta) * (delta**i) * i for i in range(I)]

In [ ]:

data = {'r_t': np.random.randint(0, 100, I)} # Example returns between -0.5 and 0.5
df = pd.DataFrame(data)

df['gains'] = df['r_t'].cumsum()
df

In [ ]:
df['r_t_hat'] = df['r_t'].ewm(com=60, adjust=False).mean() 

In [ ]:
df['std_r_t'] = df['r_t'].ewm(com=60, adjust=True).std() 

In [ ]:
df['sig'] = np.sqrt( df['std_r_t'] * 261)

In [ ]:
df

In [ ]:
df["r_t"][:25]

In [ ]:
I = 1000

t = 23

def sig_sq(t, delta, df):
    r = df['r_t']
    rh = df['r_t_hat']
    sc = 261 # Scaling constant, assuming 261 trading days in a year
    return sc * sum([(1-delta) * (delta**i) * (r[t-1-i] - rh[t])**2 for i in range(t)])


df["volat"] = [ np.sqrt(sig_sq(t, delta, df)) for t in range(0, I) ]


In [ ]:
fig = px.line(df, y="volat")
fig#.add_trace(px.line(df, y="r_t").data[0])

In [ ]:
fig = px.scatter(df, y="sig", x = "volat")
fig

In [ ]:
import pandas as pd
import numpy as np

class bla:

    def __init__(self):
        self.df = pd.read_csv("sample_df.csv", index_col='day')
        self.day = 345

    def calc_periods_returns(self, period, days):
        #data =  self.df[self.df.day == self.day] #, ['volatility', 'tic']]
        data = self.df.loc[self.day, ['date_idx', 'volatility', 'tic']].set_index('tic')

        data['ret'] = self.df.loc[self.day - days : self.day, ['ret','tic']].groupby('tic').sum()['ret']
        data['returns'] = data['ret'] / (data['volatility'] * np.sqrt(days))
        data.set_index('date_idx', inplace = True)
        self.df.loc[self.day, period] = data['returns'] 
        #print(self.df.loc[self.day])

    def doit(self, rng = [5]):
        periods = {'ret_1M': 21,
            'ret_2M': 42,
            'ret_3M': 63,
            'ret_1Y': 252} 
        for i in rng:
            self.day = i
            for p,d in periods.items():
                self.calc_periods_returns(p,d)    

In [ ]:
B = bla()
B.df

In [ ]:
B.doit(B.df.index.unique())

In [ ]:
B.df